# A walkthrough of text analysis and TF-IDF
#### Material from Google Colab Tutorials
#### Edited by Nakul Upadhya and Faisal Shaik

---
#### Note from Faisal (mrdandelion6):
I copied this file from a different repository as part of my training in data science at SUDS UofT 2024. Most of the content in these files is not written by me, I have just made some extra edits. Credit to the original authors and editors. You can see the original repository here: https://github.com/upadhyan/SUDS-2024-Bootcamp-Friday

---

We'll start by using scikit-learn to count words, then come across some of the issues with simple word count analysis. Most of these problems can be tackled with TF-IDF - a single word might mean less in a longer text, and common words may contribute less to meaning than more rare ones.

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer

pd.options.display.max_columns = 30
%matplotlib inline

## Text analysis refresher

Text analysis has a few parts. We are going to use **bag of words** analysis, which just treats a sentence like a bag of words - no particular order or anything. It's simple but it usually gets the job done adequately.

Here is our text.

In [3]:
texts = [
    "Penny bought bright blue fishes.",
    "Penny bought bright blue and orange fish.",
    "The cat ate a fish at the store.",
    "Penny went to the store. Penny ate a bug. Penny saw a fish.",
    "It meowed once at the bug, it is still meowing at the bug and the fish",
    "The cat is at the fish store. The cat is orange. The cat is meowing at the fish.",
    "Penny is a fish"
]

When you process text, you have a nice long series of steps, but let's say you're interested in three things:

1. **Tokenizing** converts all of the sentences/phrases/etc into a series of words, and then it might also include converting it into a series of numbers - math stuff only works with numbers, not words. So maybe 'cat' is 2 and 'rug' is 4 and stuff like that.
2. **Counting** takes those words and sees how many there are (obviously) - how many times does `meow` appear?
3. **Normalizing** takes the count and makes new numbers - maybe it's how many times `meow` appears vs. how many total words there are, or maybe you're seeing how often `meow` comes up to see whether it's important.

In [4]:
"Penny bought bright blue fishes".split()

['Penny', 'bought', 'bright', 'blue', 'fishes']

      Penny bought bright blue fishes.

If we **tokenize** that sentence, we're just lowercasing it, removing the punctuation and splitting on spaces - `penny bought bright blue fishes`.


The `scikit-learn` package does a **ton of stuff**, some of which includes the above. We're going to start by playing with the `CountVectorizer`.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

In [6]:
# .fit_transform will both TOKENIZE and COUNT
X = count_vectorizer.fit_transform(texts)

Let's take a look at what it found out!

In [7]:
X

<7x23 sparse matrix of type '<class 'numpy.int64'>'
	with 49 stored elements in Compressed Sparse Row format>

Okay, that looks like trash and garbage. What's a "sparse array"?????? We see that we can use `.toarray()` on a sparse array to return an `ndarray` from it.

In [8]:
X.toarray()

array([[0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0],
       [1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0],
       [0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0,
        0],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 1, 1, 1,
        1],
       [1, 2, 0, 0, 0, 0, 2, 0, 1, 0, 1, 2, 1, 1, 1, 0, 0, 0, 1, 0, 3, 0,
        0],
       [0, 2, 0, 0, 0, 0, 0, 3, 2, 0, 3, 0, 0, 1, 0, 1, 0, 0, 0, 1, 5, 0,
        0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0]], dtype=int64)

Note that `X.toarray()` returned an ndarray.

If we put on our **Computer Goggles** we see that the first sentence has the first word 3 times, the second word 1 time, the third word 1 time, etc... But we can't *read* it, really. It would look nicer as a dataframe and with the words labeled.

In [9]:
# count_vectorizer.get_feature_names_out() returns the words that were tokenized
J = pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names_out())
# this returns a pandas dataframe. Dataframe(ndarray, columns=features)
# note that data frames are a two-dimensional, data structure with labeled axes (rows and columns). see data_analysis.ipynb for more.
J

,and,at,ate,blue,bought,bright,bug,cat,fish,fishes,is,it,meowed,meowing,once,orange,penny,saw,still,store,the,to,went
0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0
2,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,2,0,0
3,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,3,1,0,1,1,1,1
4,1,2,0,0,0,0,2,0,1,0,1,2,1,1,1,0,0,0,1,0,3,0,0
5,0,2,0,0,0,0,0,3,2,0,3,0,0,1,0,1,0,0,0,1,5,0,0
6,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0


Note that any single letter words like "a" and "I" get excluded by `count_vectorizer`. If we examine the resultant matrix, we can see that there are many words which are relatively redundant and may not be useful when it comes to distinguishing meaning of the sentence (ex. and, at, it, etc.). We call these **stopwords** and we can remove the stopwords from the matrix when we vectorize our points.

In [10]:
# we'll make a new vectorizer
count_vectorizer = CountVectorizer(stop_words='english')
# count_vectorizer = CountVectorizer(stop_words=['the', 'and'])

X = count_vectorizer.fit_transform(texts)
print(count_vectorizer.get_feature_names_out())

['ate' 'blue' 'bought' 'bright' 'bug' 'cat' 'fish' 'fishes' 'meowed'
 'meowing' 'orange' 'penny' 'saw' 'store' 'went']


Note that `CounterVectorizer(stop_words='english')` will return a vectorizor that ignores stop words from English when tokenizing.

In [11]:
pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names_out())

,ate,blue,bought,bright,bug,cat,fish,fishes,meowed,meowing,orange,penny,saw,store,went
0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0
1,0,1,1,1,0,0,1,0,0,0,1,1,0,0,0
2,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0
3,1,0,0,0,1,0,1,0,0,0,0,3,1,1,1
4,0,0,0,0,2,0,1,0,1,1,0,0,0,0,0
5,0,0,0,0,0,3,2,0,0,1,1,0,0,1,0
6,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0


I still see `meowed` and `meowing` and `fish` and `fishes` - they seem the same, so let's lemmatize/stem them, ie) combine them into one occurence. Before we do that, we will introduce making our own custom tokenizer.

You can specify a `preprocessor` or a `tokenizer` when you're creating your `CountVectorizer` to do custom *stuff* on your words. Maybe we want to get rid of punctuation, lowercase things and split them on spaces (this is basically the default). `preprocessor` is supposed to return a string, so it's a little easier to work with.

In [12]:
# This is what our normal tokenizer looks like
def boring_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    # re.sub(regex, replacement, string) replaces the regex in the string with the replacement.
    # so here, we're replacing all non-alphanumeric characters (and non-hypen -) with a space. 
    return words

# example use
print(boring_tokenizer(texts[3]))
print("\n")

count_vectorizer = CountVectorizer(stop_words='english', tokenizer=boring_tokenizer)
X = count_vectorizer.fit_transform(texts)

# this should return the same thing as the previous cell because the tokenizor is how counter_vectorizer behaves by default
print(count_vectorizer.get_feature_names_out())

['penny', 'went', 'to', 'the', 'store', 'penny', 'ate', 'a', 'bug', 'penny', 'saw', 'a', 'fish']


['ate' 'blue' 'bought' 'bright' 'bug' 'cat' 'fish' 'fishes' 'meowed'
 'meowing' 'orange' 'penny' 'saw' 'store' 'went']


d:\735-D\code\synced\langs\0_misc\Learn-to-Code\python\env\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


To better understand how a tokenizer works, think of it like this: We call `count_vectorizer.fit_transform(str_list)` and `str_list` is our list of strings. Then, the tokenizer will be called on each element in `str_list`, and all the `stop_words` will be excluded. Then the filtered list of words will be used as columns and the occurence of each word in a sentence will be counted in the sparse array `X`.

Now we work on combining similar words together. We're going to use one that features a **stemmer** - something that strips the endings off of words (or tries to, at least). This one is from `nltk`.

In [13]:
# https://tartarus.org/martin/PorterStemmer/def.txt
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

print(porter_stemmer.stem('fishes'))
print(porter_stemmer.stem('meowed'))
print(porter_stemmer.stem('oranges'))
print(porter_stemmer.stem('meowing'))
print(porter_stemmer.stem('orange'))
print(porter_stemmer.stem('go'))
print(porter_stemmer.stem('went'))

fish
meow
orang
meow
orang
go
went


In [14]:
porter_stemmer = PorterStemmer()

# same thing as boring_tokenizer, except we stem the words
def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

count_vectorizer = CountVectorizer(stop_words='english', tokenizer=stemming_tokenizer)
X = count_vectorizer.fit_transform(texts)
print(count_vectorizer.get_feature_names_out())

['ate' 'blue' 'bought' 'bright' 'bug' 'cat' 'fish' 'meow' 'onc' 'orang'
 'penni' 'saw' 'store' 'went']


d:\735-D\code\synced\langs\0_misc\Learn-to-Code\python\env\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'you

Now lets look at the new version of that dataframe.

In [15]:
pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names_out())

,ate,blue,bought,bright,bug,cat,fish,meow,onc,orang,penni,saw,store,went
0,0,1,1,1,0,0,1,0,0,0,1,0,0,0
1,0,1,1,1,0,0,1,0,0,1,1,0,0,0
2,1,0,0,0,0,1,1,0,0,0,0,0,1,0
3,1,0,0,0,1,0,1,0,0,0,3,1,1,1
4,0,0,0,0,2,0,1,2,1,0,0,0,0,0
5,0,0,0,0,0,3,2,1,0,1,0,0,1,0
6,0,0,0,0,0,0,1,0,0,0,1,0,0,0


    "Penny bought bright blue fishes.",
    "Penny bought bright blue and orange fish.",
    "The cat ate a fish at the store.",
    "Penny went to the store. Penny ate a bug. Penny saw a fish.",
    "It meowed once at the bug, it is still meowing at the bug and the fish",
    "The cat is at the fish store. The cat is orange. The cat is meowing at the fish.",
    "Penny is a fish"

## TF-IDF

### Part One: Term Frequency

TF-IDF? What? It means **term frequency inverse document frequency!** It's the most important thing. Let's look at our list of phrases

1. Penny bought bright blue fishes.
2. Penny bought bright blue and orange fish.
3. The cat ate a fish at the store.
4. Penny went to the store. Penny ate a bug. Penny saw a fish.
5. It meowed once at the fish, it is still meowing at the fish. It meowed at the bug and the fish.
6. The cat is fat. The cat is orange. The cat is meowing at the fish.
7. Penny is a fish

If we are training a classifier to identify sentences about aquatic life, which is the most helpful phrase?

In [16]:
pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names_out())

,ate,blue,bought,bright,bug,cat,fish,meow,onc,orang,penni,saw,store,went
0,0,1,1,1,0,0,1,0,0,0,1,0,0,0
1,0,1,1,1,0,0,1,0,0,1,1,0,0,0
2,1,0,0,0,0,1,1,0,0,0,0,0,1,0
3,1,0,0,0,1,0,1,0,0,0,3,1,1,1
4,0,0,0,0,2,0,1,2,1,0,0,0,0,0
5,0,0,0,0,0,3,2,1,0,1,0,0,1,0
6,0,0,0,0,0,0,1,0,0,0,1,0,0,0


Probably the one where `fish` appears two times.

    It meowed once at the fish, it is still meowing at the fish. It meowed at the bug and the fish.
    
But are all the others the same?

    Penny is a fish.

    Penny went to the store. Penny ate a bug. Penny saw a fish.

In the second one we spend less time talking about the fish. Think about a huge long document where they say your name once, versus a tweet where they say your name once. Which one are you more important in? Probably the tweet, since you take up a larger percentage of the text.

This is **term frequency** - taking into account how often a term shows up. We're going to take this into account by using the `TfidfVectorizer` in the same way we used the `CountVectorizer`.

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

We're going to start by ignoring idf (inverse document frequency) for now. We do this by setting `use_idf=False`. 

In [18]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', tokenizer=stemming_tokenizer, use_idf=False, norm='l1')
X = tfidf_vectorizer.fit_transform(texts)
pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

d:\735-D\code\synced\langs\0_misc\Learn-to-Code\python\env\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
d:\735-D\code\synced\langs\0_misc\Learn-to-Code\python\env\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'som

,ate,blue,bought,bright,bug,cat,fish,meow,onc,orang,penni,saw,store,went
0,0.000000,0.200000,0.200000,0.200000,0.000000,0.000,0.200000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000
1,0.000000,0.166667,0.166667,0.166667,0.000000,0.000,0.166667,0.000000,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000
2,0.250000,0.000000,0.000000,0.000000,0.000000,0.250,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000
3,0.111111,0.000000,0.000000,0.000000,0.111111,0.000,0.111111,0.000000,0.000000,0.000000,0.333333,0.111111,0.111111,0.111111
4,0.000000,0.000000,0.000000,0.000000,0.333333,0.000,0.166667,0.333333,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.375,0.250000,0.125000,0.000000,0.125000,0.000000,0.000000,0.125000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000


Now our numbers have shifted a little bit. Instead of just being a count, it's *the percentage of the words*.

    value = (number of times word appears in sentence) / (number of words in sentence)

After we remove the stopwords, the term `fish` is 50% of the words in `Penny is a fish` vs. 37.5% in `It meowed once at the fish, it is still meowing at the fish. It meowed at the bug and the fish.`.

> **Note:** We made it be the percentage of the words by passing in `norm="l1"` - by default it's normally an L2 (Euclidean) norm, which is actually better, but I thought it would make more sense starting by using the L1 - a.k.a. terms divided by words -norm.

So now when we train a classifier, it will be able to identify the important words easier because our pre-processing takes into account whether half of our words are `fish` or 1% of millions upon millions of words is `fish`. But we aren't done yet! We will now explore inverse document frequnecy as well.

### Part Two: Inverse document frequency

In addition to its importance to a given sentence, we also want to give information on how prevelant a given token is in the dataset as a whole. A token that is mentioned a lot in every dataset will probably be less important than a token that appears infrequently.

This is **inverse document frequency** - the more often a term shows up across *all* documents, the less important it is in our matrix.

In [30]:
# use_idf=True is default, but we'll include it here for clarity
idf_vectorizer = TfidfVectorizer(stop_words='english', tokenizer=stemming_tokenizer, use_idf=True, norm='l1')
X = idf_vectorizer.fit_transform(texts)
idf_df = pd.DataFrame(X.toarray(), columns=idf_vectorizer.get_feature_names_out())
idf_df

d:\735-D\code\synced\langs\0_misc\Learn-to-Code\python\env\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
d:\735-D\code\synced\langs\0_misc\Learn-to-Code\python\env\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'som

,ate,blue,bought,bright,bug,cat,fish,meow,onc,orang,penni,saw,store,went
0,0.000000,0.235463,0.235463,0.235463,0.000000,0.000000,0.118871,0.000000,0.000000,0.000000,0.174741,0.000000,0.000000,0.000000
1,0.000000,0.190587,0.190587,0.190587,0.000000,0.000000,0.096216,0.000000,0.000000,0.190587,0.141437,0.000000,0.000000,0.000000
2,0.297654,0.000000,0.000000,0.000000,0.000000,0.297654,0.150267,0.000000,0.000000,0.000000,0.000000,0.000000,0.254425,0.000000
3,0.125073,0.000000,0.000000,0.000000,0.125073,0.000000,0.063142,0.000000,0.000000,0.000000,0.278455,0.150675,0.106908,0.150675
4,0.000000,0.000000,0.000000,0.000000,0.350291,0.000000,0.088420,0.350291,0.210997,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.437035,0.147088,0.145678,0.000000,0.145678,0.000000,0.000000,0.124521,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.404858,0.000000,0.000000,0.000000,0.595142,0.000000,0.000000,0.000000


Note that this time we enter `use_idf=True`. This is already True by default but it's added in for emphasis as we had set it to False before.

Notice how 'meow' increased in value because it's an infrequent term, and `fish` dropped in value because it's so frequent.

That meowing one (index 4) has gone from `0.50` to `0.43`, while `Penny is a fish` (index 6) has dropped to `0.40`. Now hooray, the meowing one is going to show up earlier when searching for "fish meow" because *fish shows up all of the time, so we want to ignore it a lil' bit*.

Let's try changing it to `norm='l2'` (or just removing `norm` completely) to see how this might change our matrix.

In [20]:
# norm gets set to l2 by default
l2_vectorizer = TfidfVectorizer(stop_words='english', tokenizer=stemming_tokenizer, use_idf=True)
X = l2_vectorizer.fit_transform(texts)
l2_df = pd.DataFrame(X.toarray(), columns=l2_vectorizer.get_feature_names_out())
l2_df

d:\735-D\code\synced\langs\0_misc\Learn-to-Code\python\env\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
d:\735-D\code\synced\langs\0_misc\Learn-to-Code\python\env\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'som

,ate,blue,bought,bright,bug,cat,fish,meow,onc,orang,penni,saw,store,went
0,0.000000,0.512612,0.512612,0.512612,0.000000,0.000000,0.258786,0.000000,0.000000,0.000000,0.380417,0.000000,0.000000,0.000000
1,0.000000,0.456170,0.456170,0.456170,0.000000,0.000000,0.230292,0.000000,0.000000,0.456170,0.338530,0.000000,0.000000,0.000000
2,0.578752,0.000000,0.000000,0.000000,0.000000,0.578752,0.292176,0.000000,0.000000,0.000000,0.000000,0.000000,0.494698,0.000000
3,0.303663,0.000000,0.000000,0.000000,0.303663,0.000000,0.153301,0.000000,0.000000,0.000000,0.676058,0.365821,0.259561,0.365821
4,0.000000,0.000000,0.000000,0.000000,0.641958,0.000000,0.162043,0.641958,0.386682,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.840166,0.282766,0.280055,0.000000,0.280055,0.000000,0.000000,0.239382,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.562463,0.000000,0.000000,0.000000,0.826823,0.000000,0.000000,0.000000


Note that our values all went up drastically.

## Real Life Example

Lets take what we learned and apply it to a real life classification task. AG is a collection of more than 1 million news articles organized into four different topics: Business, Tech, Sports, and World News.
Our task is to:
1. Apply the text pre-processing you learnt above to get a clean dataset.
2. Train a classifier on your processed data.
3. Report the accuracy, precision, recall, and F1 (there are multiple classes so think about which F1 to report)

Code to download and sample the data is already provided.



In [24]:
from datasets import load_dataset

dataset = load_dataset("ag_news", cache_dir='data/datasets/text/ag_news')
train = dataset['train']
test = dataset['test']
# convert to pandas
train = pd.DataFrame(train).head(2000)
test = pd.DataFrame(test).head(500)
print(train.shape)
print(test.shape)
train.head()

(2000, 2)
(500, 2)


,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2


Note that train is our training dataset and test is ourr testing data set.

In [36]:
train_texts = train['text'] # get the column (this is a Pandas Series)
test_texts = test['text'] 

# construct our vectorizer just like before with the same tokenizer. recall the tokenizer just filters our words
tfidf_vectorizer = TfidfVectorizer(stop_words='english', 
                                   tokenizer=stemming_tokenizer, 
                                   use_idf=True, 
                                   norm='l2',
                                   min_df=0.01,
                                   max_df=0.99)
# set min_df and max_df to filter out words that appear in less than 1% of the documents or more than 99% of the documents

X_train = tfidf_vectorizer.fit_transform(train_texts).toarray()

# this time we use tfidf_vectorizer.transform()
# we apply a vectorization to the test data using the IDFs for each word which was obtained from the train data
X_test = tfidf_vectorizer.transform(test_texts).toarray()


Y_train = train['label']
Y_test = test['label']

train_df = pd.DataFrame(data=X_train, columns=tfidf_vectorizer.get_feature_names_out())
test_df = pd.DataFrame(data=X_test, columns=tfidf_vectorizer.get_feature_names_out())

d:\735-D\code\synced\langs\0_misc\Learn-to-Code\python\env\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
d:\735-D\code\synced\langs\0_misc\Learn-to-Code\python\env\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'som

In [37]:
train_df

,-,--,0,000,1,10,100,15,16,17,2,2004,3,36,39,...,week,west,win,window,wireless,women,won,work,world,worri,www,xp,year,yesterday,york
0,0.135449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1,0.101354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
2,0.073639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.173042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.224442,0.0,0.0,0.0,0.0,0.000000
3,0.082500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
4,0.078498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.172364,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.090436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1996,0.137244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1997,0.109065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1998,0.123393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000


In [38]:
test_df

,-,--,0,000,1,10,100,15,16,17,2,2004,3,36,39,...,week,west,win,window,wireless,women,won,work,world,worri,www,xp,year,yesterday,york
0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.065555,0.130290,0.0,0.0,0.0,0.192245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.199803,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.105762,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.241596,0.0,0.0,0.0,0.265344,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.098530,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.130323,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.000000,0.144059,0.0,0.0,0.0,0.212561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
496,0.000000,0.144241,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.202776,0.0
497,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
498,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


We have finished cleaning up our training texts. Now we will train a classifier on this data. We will use `xgboost` libraryr which is provides a gradient boosting framework. 

### Gradient Boosting
Gradient boosting is an ML technique thaat produces a prediction model in the form of an esemble of weak prerdiction models, typically decision trees. It builds the model in a stage-wise fashion and generalizes them by allowing optimization of abitrary differnetiable loss function.

XGBoost is a go-to library for high performance machine learning tasks, especially where accuracy is a critical criterion. It supports various objection functions, including regression, classification, and ranking.

In [39]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score

In [ ]:
hyperparameter_search = {
    'max_depth': [2,6], 
    'lambda': [0,0, 1e-3],
    'alpha': [0,0,1e-3] 
}

evaluation_metric = make_scorer(accuracy_score, greater_is_better=True)